---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [0]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [0]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [0]:
from google.colab import drive
drive.mount('/data')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /data


In [0]:
!pip install xlrd

    100% |████████████████████████████████| 112kB 2.5MB/s 


In [0]:
import pandas as pd
import numpy as np
import re 
from scipy.stats import ttest_ind
def prettity(x):
  m = re.search('\[',x)
  if m:
    x=x[:m.start()]
  m=re.search('\(',x)
  if m:
    x=x[m.start():]

  return x
def prettitya(x):
  m = re.search('\(',x)
  if m:
    x=x[:m.start()]
  return x
file = open('/data/My Drive/data/university_towns.txt', 'r') 
pds=pd.Series([0,0],index=['a','b'])
n=""
for i in file:
  if('[edit]' in i):
    n=i
    continue
  else:
    a=pd.Series([i,n],index=['a','b'])
    pds=pd.concat([pds,a],axis=1,join='inner')
pds

In [0]:
def get_list_of_university_towns():
    data = []
    State = None
    State_RegionNames = []
    with open('/data/My Drive/data/university_towns.txt') as file:
        for line in file:
            thisLine = line[:-1]
            if thisLine[-6:] == '[edit]':
                State = thisLine[:-6]
                continue
            if '(' in line:
                RegionName = thisLine[:thisLine.index('(')-1]
                State_RegionNames.append([State,RegionName])
            else:
                RegionName = thisLine
                State_RegionNames.append([State,RegionName])
            data.append(thisLine)
    df = pd.DataFrame(State_RegionNames,columns = ['State','RegionName'])
    return df
  
get_list_of_university_towns()

,State,RegionName
100,Illinois,Evanston


In [0]:
import pandas as pd
def get_recession_start():
    quarter = pd.read_excel('/data/My Drive/data/gdplev.xls', skiprows=219)
    quarter = quarter[['1999q4', 9926.1]]
    quarter.columns = ['Quarter','GDP']
    for i in range(2, len(quarter)):
        if (quarter.iloc[i-2][1] - quarter.iloc[i-1][1])>0 and (quarter.iloc[i-1][1] - quarter.iloc[i][1])>0:
            return quarter.iloc[i-2][0]

get_recession_start()

In [0]:
def get_recession_end():
    quarter = pd.read_excel('/data/My Drive/data/gdplev.xls', skiprows=219)
    quarter = quarter[['1999q4', 9926.1]]
    quarter.columns = ['Quarter','GDP']
    recession_start = get_recession_start()
    recession_start_index = quarter[quarter['Quarter'] == recession_start].index.tolist()[0]
    quarter=quarter.iloc[recession_start_index:]
    for i in range(2, len(quarter)):
        if (quarter.iloc[i-2][1] - quarter.iloc[i-1][1])<0 and (quarter.iloc[i-1][1] - quarter.iloc[i][1])<0:
            return quarter.iloc[i][0]
get_recession_end()

In [0]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    quarter = pd.read_excel('/data/My Drive/data/gdplev.xls', skiprows=219)
    quarter = quarter[['1999q4', 9926.1]]
    quarter.columns = ['Quarter','GDP'] 
    recession_start = get_recession_start()
    recession_start_index = quarter[quarter['Quarter'] == recession_start].index.tolist()[0]
    recession_end = get_recession_end()
    recession_end_index = quarter[quarter['Quarter'] == recession_end].index.tolist()[0]
    quarter=quarter.iloc[recession_start_index:recession_end_index+1]
    recession_bottom = quarter['GDP'].min()
    recession_bottom_index = quarter[quarter['GDP'] == recession_bottom].index.tolist()[0]-recession_start_index
    return quarter.iloc[recession_bottom_index]['Quarter']
get_recession_bottom()

In [0]:
def new_col_names():
    years = list(range(2000,2017))
    quars = ['q1','q2','q3','q4']
    quar_years = []
    for i in years:
        for x in quars:
            quar_years.append((str(i)+x))
    return quar_years[:67]
new_col_names()
def convert_housing_data_to_quarters():
    data = pd.read_csv('/data/My Drive/data/City_Zhvi_AllHomes.csv')
    data.drop(columns=['Metro','CountyName','RegionID','SizeRank'],axis=1,inplace=True)
    data['State'] = data['State'].map(states)
    data.set_index(['State','RegionName'],inplace=True)
    col = list(data.columns)
    col = col[0:45]
    data.drop(col,axis=1,inplace=True)

    #qs is the quarters of the year
    qs = [list(data.columns)[x:x+3] for x in range(0, len(list(data.columns)), 3)]
    
    # new columns
    column_names = new_col_names()
    for col,q in zip(column_names,qs):
        data[col] = data[q].mean(axis=1)
        
    data = data[column_names]
    return data
convert_housing_data_to_quarters()

In [0]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"